# Deploy a Keras or TensorFlow model trained anywhere using Amazon SageMaker


Amazon SageMaker makes it easier for any developer or data scientist to build, train, and deploy machine learning (ML) models. While it’s designed to alleviate the undifferentiated heavy lifting from the full life cycle of ML models, Amazon SageMaker’s capabilities can also be used independently of one another; that is, models trained in Amazon SageMaker can be optimized and deployed outside of Amazon SageMaker (or even out of the cloud on mobile or IoT devices at the edge). Conversely, Amazon SageMaker can deploy and host pre-trained models from model zoos, or other members of your team. 

In this blog post, we’ll demonstrate how to deploy a trained Keras (TensorFlow or MXNet backend) or TensorFlow model using Amazon SageMaker, taking advantage of Amazon SageMaker deployment capabilities, such as selecting the type and number of instances, performing A/B testing, and Auto Scaling.  Auto Scaling clusters are spread across multiple Availability Zones to deliver high performance and high availability.

Your trained model will need to be saved in either the Keras (JSON and weights hdf5) format or the TensorFlow Protobuf format. If you’d like to begin from a sample notebook that supports this blog post, [download it here](https://s3.amazonaws.com/aws-ml-blog/artifacts/keras-tensorflow-model-deployment/keras-tensorflow-model-deployment.ipynb).

For more on training the model on SageMaker and deploying, refer to https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb

### Step 1. Set up

In the AWS Management Console, go to the Amazon SageMaker console. Choose Notebook Instances, and create a new notebook instance. Upload the current notebook and set the kernel to conda_tensorflow_p36.

The get_execution_role function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

In [1]:

from sagemaker import Session

session = Session(boto_session=boto3.Session(region_name="us-east-1"))
role = session.get_execution_role()


NameError: name 'boto3' is not defined

In [2]:
pip list


Package               Version
--------------------- -----------
absl-py               2.1.0
aiofiles              22.1.0
aiosqlite             0.19.0
anyio                 3.7.1
argon2-cffi           23.1.0
argon2-cffi-bindings  21.2.0
arrow                 1.2.3
astor                 0.8.1
attrs                 23.2.0
Babel                 2.14.0
backcall              0.2.0
beautifulsoup4        4.12.3
bleach                6.0.0
boto3                 1.33.13
botocore              1.33.13
cached-property       1.5.2
certifi               2024.2.2
cffi                  1.15.1
charset-normalizer    3.3.2
click                 8.1.7
cloudpickle           2.2.1
colorama              0.4.6
cycler                0.11.0
debugpy               1.7.0
decorator             5.1.1
defusedxml            0.7.1
dill                  0.3.7
entrypoints           0.4
exceptiongroup        1.2.0
fastjsonschema        2.19.1
Flask                 2.2.5
fonttools             4.38.0
fqdn                  1.

In [3]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

ValueError: Must setup local AWS configuration with a region supported by SageMaker.

### Step 2. Load the Keras model using the json and weights file

If you saved your model in the TensorFlow ProtoBuf format, skip to "Step 4. Convert the TensorFlow model to an Amazon SageMaker-readable format.

In [ ]:
import keras
from keras.models import model_from_json

Create a directory called keras_model, navigate to keras_model from the Jupyter notebook home, and upload the model.json and model-weights.h5 files (using the "Upload" menu on the Jupyter notebook home).

In [ ]:
!mkdir keras_model

Navigate to keras_model from the Jupyter notebook home, and upload your model.json and model-weights.h5 files (using the "Upload" menu on the Jupyter notebook home). To use a sample model for this exercise [download and unzip the files found here](https://s3.amazonaws.com/aws-ml-blog/artifacts/keras-tensorflow-model-deployment/model.zip), then upload them to keras_model.

In [ ]:
!ls keras_model

In [ ]:
json_file = open('/home/ec2-user/SageMaker/keras_model/'+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/model-weights.h5')
print("Loaded model from disk")

### Step 3. Export the Keras model to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [ ]:
# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [ ]:
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

In [ ]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

In [ ]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

### Step 4. Convert TensorFlow model to a SageMaker readable format

Move the TensorFlow exported model into a directory export\Servo\. SageMaker will recognize this as a loadable TensorFlow model. Your directory and file structure should look like:

In [ ]:
!ls export

In [ ]:
!ls export/Servo

In [ ]:
!ls export/Servo/1

In [ ]:
!ls export/Servo/1/variables

####  Tar the entire directory and upload to S3

In [ ]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

### Step 5. Deploy the trained model

The entry_point file "train.py" can be an empty Python file. The requirement will be removed at a later date.

In [ ]:
!touch train.py

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

Note: You will need to update the endpoint in the command below with the endpoint name from the output of the previous cell (INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-2018-09-28-14-26-05-826)

In [ ]:
endpoint_name = 'sagemaker-tensorflow-2019-01-29-21-22-40-206'

In [ ]:
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

### Step 6. Invoke the endpoint

#### Invoke the SageMaker endpoint from the notebook

In [ ]:
import numpy as np

# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 50)
predictor.predict(data)

#### Invoke the SageMaker endpoint using a boto3 client

In [ ]:
import json
import boto3
import numpy as np
import io
 
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 50).tolist()
response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data))
response_body = response['Body']
print(response_body.read())

### Step 7. Clean up

To avoid incurring unnecessary charges, use the AWS Management Console to delete the resources that you created for this exercise:
https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

### Conclusion

In this blog post, we demonstrated deploying a trained Keras or TensorFlow model at scale using Amazon SageMaker, independent of the computing resource used for model training. This gives you the flexibility to use your existing workflows for model training, while easily deploying the trained models to production with all the benefits offered by a managed platform. These benefits include the ability to select the optimal type and number of deployment instances, perform A/B testing, and auto scale. The Auto Scaling clusters of Amazon SageMaker ML instances can be spread across multiple Availability Zones to deliver both high performance and high availability.

### About the author

Dr. Priya Ponnapalli is a principal data scientist at Amazon ML Solutions Lab, where she helps AWS customers across different industries accelerate their AI and cloud adoption.